In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imblearn as debias # https://imbalanced-learn.org/stable/

In [11]:
# RANDOM SAMPLING
def XYsplit(df: pd.DataFrame) -> list:
  """Function to split a df into X and Y arrays."""
  Y = df["diagnosis"]
  X = df.drop(["diagnosis"], axis = 1)
  return [X, Y]

def rndSample(X, Y, mode = "under") -> list:
  """Random Undersampling Technique"""
  sampler = None
  if mode == "under": sampler = debias.under_sampling.RandomUnderSampler(random_state = 0)
  elif mode == "over": sampler = debias.over_sampling.RandomOverSampler(random_state = 0)
  Xresampled,Yresampled = sampler.fit_resample(X, Y)
  return [Xresampled, Yresampled]

In [ ]:
# Example driver
from ipynb.fs.full.preprocessing import dataPreProcessing
from collections import Counter

vars = dataPreProcessing(dsFile = "data", processes = ["clean", "predMap"])
df = vars["df"]
X, Y = XYsplit(df)
newX, newY = rndSample(X, Y, mode = "under")
print(sorted(Counter(newY).items()))


NameError: name 'Counter' is not defined